# OptimizingDelivery Import and Preparation 

## Import libraries

In [74]:
import os 
import pandas as pd

## Import data

In [75]:
%store -r optimizingdelivery_source_dir
%store -r optimizingdelivery_data_dir

In [76]:
# Define the list of file names
file_names = ['dim_customers.csv', 
              'dim_date.csv', 
              'dim_products.csv', 
              'dim_targets_orders.csv', 
              'fact_order_lines.csv', 
              'fact_orders_aggregate.csv']

# Read each CSV file into a pandas DataFrame
og_dataframes = [pd.read_csv(os.path.join(optimizingdelivery_source_dir, file)) for file in file_names]

# Unpack the list of DataFrames into individual variables
customers, all_dates, products, target_orders, order_lines, orders_aggregate = og_dataframes

In [77]:
print(customers.shape)
print(all_dates.shape)
print(products.shape)
print(target_orders.shape)
print(order_lines.shape)
print(orders_aggregate.shape)

(35, 3)
(183, 3)
(18, 3)
(35, 4)
(57096, 11)
(31729, 6)


## Prepare data

### Duplicates and nans

In [78]:
# Define dataframes and dataframe names
dataframes = [customers, all_dates, products, target_orders, order_lines, orders_aggregate]
dataframe_names = ['customers', 'all_dates', 'products', 'target_orders', 'order_lines', 'orders_aggregate']

In [79]:
# Check dataframes for duplicate rows

# Define function to check for duplicate rows
def check_dupe(dataframe_names, dataframes):

    # Loop through each dataframe
    for df_name, df in zip(dataframe_names, dataframes):

        # Find duplicate rows
        duplicates = df[df.duplicated()]

        # Yield the dataframe name and duplicated rows
        yield df_name, duplicates if not duplicates.empty else pd.DataFrame()

# Output the duplicate rows for each dataframe
for df_name, duplicates in check_dupe(dataframe_names, dataframes):
    
    # Print 
    print(f"{df_name}:")
    if not duplicates.empty:
        print(duplicates)
    else:
        print("none")


customers:
none
all_dates:
none
products:
none
target_orders:
none
order_lines:
none
orders_aggregate:
none


In [80]:
# Check dataframes for nan rows

# Define function to check for nan rows
def check_nan(dataframe_names, dataframes):

    # Loop through each dataframe
    for df_name, df in zip(dataframe_names, dataframes):

        # Find rows with nan values
        nan_rows = df[df.isnull().any(axis=1)]

        # Yield the dataframe name and rows with nan values
        yield df_name, nan_rows if not nan_rows.empty else None

# Output the rows with nan values for each dataframe or output none
for df_name, nan_rows in check_nan(dataframe_names, dataframes):

    # Print 
    print(f"{df_name}:")
    if nan_rows is not None:
        print(nan_rows)
    else:
        print("none")

customers:
none
all_dates:
none
products:
none
target_orders:
none
order_lines:
none
orders_aggregate:
none


### Refine datatypes

#### customers

In [81]:
customers.dtypes

customer_id       int64
customer_name    object
city             object
dtype: object

#### all_dates

In [82]:
# Create new dates dataframe so it encompasses all of 2022
dates_2022 = pd.DataFrame({'date': pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')})

# Week number in source data is 8 days ahead of default week numbers
# Make sure new week number matchs with source data, testing with 'boingo'
dates_2022['boingo'] = dates_2022['date'].dt.isocalendar().week.shift(-8)

# Fill in nans from shifting
dates_2022.loc[dates_2022.index == 357, 'boingo'] = 52
dates_2022.loc[dates_2022.index > 357, 'boingo'] = 53

In [83]:
# Merge dataframes to check new column matches source

# prepare source for merge
all_dates['date'] = pd.to_datetime(all_dates['date'], format='%d-%b-%y')
all_dates['week_number'] = all_dates['week_no'].str.split().str[1].astype(int)

# Merge
merge = pd.merge(all_dates, dates_2022, how='left', on='date')

# Output whether rows match
if len(merge[~(merge['week_number'] == merge['boingo'])]) == 0:
    print("all rows match")

all rows match


In [84]:
# Create new columns
dates_2022['month'] = dates_2022['date'].dt.month.astype('int64')
dates_2022['day'] = dates_2022['date'].dt.day.astype('int64')
dates_2022['week_number'] = dates_2022['boingo'].astype('int64')

# Reorder columns
dates_2022 = dates_2022[['date', 'month', 'day', 'week_number']]

In [85]:
dates_2022.dtypes

date           datetime64[ns]
month                   int64
day                     int64
week_number             int64
dtype: object

#### products

In [86]:
products.dtypes

product_name    object
product_id       int64
category        object
dtype: object

#### target_orders

In [87]:
target_orders.dtypes

customer_id       int64
ontime_target%    int64
infull_target%    int64
otif_target%      int64
dtype: object

#### order_lines

In [88]:
order_lines.dtypes

order_id                object
order_placement_date    object
customer_id              int64
product_id               int64
order_qty                int64
agreed_delivery_date    object
actual_delivery_date    object
delivery_qty             int64
In Full                  int64
On Time                  int64
On Time In Full          int64
dtype: object

In [89]:
# Change datatypes
order_lines['order_placement_date'] = pd.to_datetime(order_lines['order_placement_date'], format='%A, %B %d, %Y')
order_lines['agreed_delivery_date'] = pd.to_datetime(order_lines['agreed_delivery_date'], format='%A, %B %d, %Y')
order_lines['actual_delivery_date'] = pd.to_datetime(order_lines['actual_delivery_date'], format='%A, %B %d, %Y')

# Sort dataframe by date increasing
order_lines = order_lines.sort_values('order_placement_date').reset_index(drop=True)

# Rename columns
order_lines = order_lines.rename(columns={
                                          'In Full': 'in_full',
                                          'On Time': 'on_time',
                                          'On Time In Full': 'on_time_in_full'
})

# Reorder columns
order_lines = order_lines[['order_id', 'order_placement_date', 'customer_id', 'product_id',
       'order_qty', 'agreed_delivery_date', 'actual_delivery_date',
       'delivery_qty', 'in_full', 'on_time', 'on_time_in_full']].copy()

In [90]:
order_lines.dtypes

order_id                        object
order_placement_date    datetime64[ns]
customer_id                      int64
product_id                       int64
order_qty                        int64
agreed_delivery_date    datetime64[ns]
actual_delivery_date    datetime64[ns]
delivery_qty                     int64
in_full                          int64
on_time                          int64
on_time_in_full                  int64
dtype: object

#### orders_aggregate

In [91]:
orders_aggregate.dtypes

order_id                object
customer_id              int64
order_placement_date    object
on_time                  int64
in_full                  int64
otif                     int64
dtype: object

In [92]:
orders_aggregate['order_placement_date'] = pd.to_datetime(orders_aggregate['order_placement_date'], format='%d-%b-%y')


In [93]:
orders_aggregate.dtypes

order_id                        object
customer_id                      int64
order_placement_date    datetime64[ns]
on_time                          int64
in_full                          int64
otif                             int64
dtype: object

### Add calculated columns

In [94]:
order_lines.shape

(57096, 11)

In [95]:
print(order_lines.shape)
order_lines.dtypes

(57096, 11)


order_id                        object
order_placement_date    datetime64[ns]
customer_id                      int64
product_id                       int64
order_qty                        int64
agreed_delivery_date    datetime64[ns]
actual_delivery_date    datetime64[ns]
delivery_qty                     int64
in_full                          int64
on_time                          int64
on_time_in_full                  int64
dtype: object

In [96]:
list(order_lines.columns)

['order_id',
 'order_placement_date',
 'customer_id',
 'product_id',
 'order_qty',
 'agreed_delivery_date',
 'actual_delivery_date',
 'delivery_qty',
 'in_full',
 'on_time',
 'on_time_in_full']

#### Line Fill Rate

In [97]:
# Create line_fill_rate_pct column

order_lines['line_fill_rate_pct'] = round(order_lines.groupby('order_id')['in_full'].transform('mean') * 100, 2)

In [98]:
# Check line_fill_rate_pct

order_lines[order_lines['order_id'].duplicated(keep=False)].sort_values('order_id')[['order_id', 'product_id', 'in_full', 'line_fill_rate_pct']].head(20)

,order_id,product_id,in_full,line_fill_rate_pct
11987,FAP410101302,25891103,1,66.67
12128,FAP410101302,25891203,1,66.67
12141,FAP410101302,25891302,0,66.67
11665,FAP410101402,25891201,1,66.67
11648,FAP410101402,25891402,0,66.67
11630,FAP410101402,25891101,1,66.67
12332,FAP410101502,25891501,1,100.00
12397,FAP410101502,25891502,1,100.00
12421,FAP410101502,25891303,1,100.00
11908,FAP410102503,25891202,0,33.33


#### Volume fill rate

In [99]:
# Create volume_fill_rate_pct column

order_lines['volume_fill_rate_pct'] = round(order_lines.groupby('order_id')['delivery_qty'].transform('sum') / order_lines.groupby('order_id')['order_qty'].transform('sum') * 100, 2)

In [100]:
# Check volume_fill_rate_pct

order_lines[order_lines['order_id'].duplicated(keep=False)].sort_values('order_id')[['order_id', 'product_id', 'order_qty', 'delivery_qty', 'volume_fill_rate_pct']].head(20)

,order_id,product_id,order_qty,delivery_qty,volume_fill_rate_pct
11987,FAP410101302,25891103,493,493,99.78
12128,FAP410101302,25891203,374,374,99.78
12141,FAP410101302,25891302,46,44,99.78
11665,FAP410101402,25891201,442,442,94.30
11648,FAP410101402,25891402,299,239,94.30
11630,FAP410101402,25891101,311,311,94.30
12332,FAP410101502,25891501,123,123,100.00
12397,FAP410101502,25891502,142,142,100.00
12421,FAP410101502,25891303,23,23,100.00
11908,FAP410102503,25891202,253,228,89.26


### Rename and reorder columns

In [ ]:
# Rename columsn for consistency
customer_split.rename(columns={ 'infull_target%': 'if_target_pct',
                                'ontime_target%': 'ot_target_pct', 
                                'otif_target%': 'otif_target_pct'}, inplace=True)

In [ ]:
# Reorgnize columns for readability
customer_split = customer_split[['customer_id', 'if_pct', 'if_target_pct', 
                                 'ot_pct', 'ot_target_pct', 
                                 'otif_pct', 'otif_target_pct']]

## Save prepared data

In [ ]:
# Save prepared data to parquet and csv files

# Define dataframes and dataframe names
dataframes = [customers, dates_2022, products, target_orders, order_lines, orders_aggregate]
dataframe_names = ['customers', 'dates_2022', 'products', 'target_orders', 'order_lines', 'orders_aggregate']

# Parquet
# Iterate over dataframes and names
for dataframe, name in zip(dataframes, dataframe_names):

    # Create file paths
    file_path = os.path.join(optimizingdelivery_data_dir, f"{name}.parquet")
    
    # Save the dataframes
    dataframe.to_parquet(file_path)

# CSV
# Iterate over dataframes and names
for dataframe, name in zip(dataframes, dataframe_names):

    # Create file paths
    file_path = os.path.join(optimizingdelivery_data_dir, f"{name}.csv")
    
    # Save the dataframes
    dataframe.to_csv(file_path)